In [1]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.pipeline import make_pipeline, make_union
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from tpot import *
from xgboost import *
from sklearn.ensemble import *
from sklearn.svm import *


/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


# DATASET : Owner median < 50 k / Jeu > 1 €

In [2]:
df = pd.read_csv('../raw_data/data_50k.csv')

In [3]:
df.head()

,steam_appid,owner_median,release_date,name,top_5_tags,topic_0,topic_1,topic_2,topic_3,topic_4,...,english,price,has_a_website,nb_review,owner_estimated,rating,popularity,score_descriptif,average_playtime,median_playtime
0,7340,10000.0,2010-04-21,Azada,casual puzzle 1980s relaxing realistic,0.015666,0.307211,0.015666,0.015666,0.015666,...,1,6.99,1,12,2496.0,7.012399,9.19781,1,0,0
1,50910,10000.0,2010-04-21,Professor Fizzwizzle and the Molten Mystery,casual puzzle 1980s relaxing realistic,0.015666,0.307211,0.015666,0.015666,0.015666,...,1,4.99,1,12,2496.0,7.012399,9.19781,1,0,0
2,50920,10000.0,2010-04-21,Hidden Expedition: Amazon,casual hidden_object 1980s realistic real_time...,0.016137,0.016137,0.016137,0.016137,0.016137,...,1,6.99,1,12,2496.0,7.012399,9.19781,6,0,0
3,50930,10000.0,2010-04-21,Hidden Expedition: Everest,casual hidden_object 1980s realistic real_time...,0.016137,0.016137,0.016137,0.016137,0.016137,...,1,6.99,1,5,1040.0,7.012399,9.19781,4,0,0
4,50940,10000.0,2010-04-21,Hidden Expedition: Titanic,casual hidden_object 1980s realistic real_time...,0.016137,0.016137,0.016137,0.016137,0.016137,...,1,6.99,1,6,1248.0,7.012399,9.19781,5,0,0


In [4]:
df.shape

(17420, 35)